<a href="https://colab.research.google.com/github/markumreed/colab_pyspark/blob/main/pyspark_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

The following three cells must be ran in order to use PySpark in Google Colab. 

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

# Spark DataFrame Basics

Spark DataFrames allow for easy handling of large datasets. 

* Easy syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs

## Create a DataFrame


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("pyspark_basics").getOrCreate()

In [ ]:
%%writefile user_simple.json
{"name":"Bob"}
{"name":"Jim", "age":40}
{"name":"Mary", "age": 24}

Writing user_simple.json


In [ ]:
df = spark.read.json("user_simple.json")

In [ ]:
df

DataFrame[age: bigint, name: string]

## Show DataFrame


In [ ]:
df.show()

+----+----+
| age|name|
+----+----+
|null| Bob|
|  40| Jim|
|  24|Mary|
+----+----+



In [ ]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
df.columns

['age', 'name']

In [ ]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [ ]:
df.describe().show()

+-------+------------------+----+
|summary|               age|name|
+-------+------------------+----+
|  count|                 2|   3|
|   mean|              32.0|null|
| stddev|11.313708498984761|null|
|    min|                24| Bob|
|    max|                40|Mary|
+-------+------------------+----+



## Specifying Schema Structure

- Some data types make it easier to infer schema. 

- Often have to set the schema yourself

- Spark has tools to help specify the structure

Next we need to create the list of Structure fields
  * :param name: string, name of the field.
  * :param dataType: :class:`DataType` of the field.
  * :param nullable: boolean, whether the field can be null (None) 

In [ ]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [ ]:
data_schema = [StructField("age", IntegerType(), True), StructField("name",StringType(), True)]

In [ ]:
final_struc = StructType(fields=data_schema)

In [ ]:
df = spark.read.json("user_simple.json", schema=final_struc)

In [ ]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
df.show()

+----+----+
| age|name|
+----+----+
|null| Bob|
|  40| Jim|
|  24|Mary|
+----+----+



## Grab Data

In [ ]:
df['age']

Column<b'age'>

In [ ]:
type(df['age'])

pyspark.sql.column.Column

In [ ]:
df.select("age")

DataFrame[age: int]

In [ ]:
type(df.select("age"))

pyspark.sql.dataframe.DataFrame

In [ ]:
df.select("age").show()

+----+
| age|
+----+
|null|
|  40|
|  24|
+----+



In [ ]:
df.head(2)

[Row(age=None, name='Bob'), Row(age=40, name='Jim')]

In [ ]:
df.select(["name","age"])

DataFrame[name: string, age: int]

In [ ]:
df.select(["name","age"]).show()

+----+----+
|name| age|
+----+----+
| Bob|null|
| Jim|  40|
|Mary|  24|
+----+----+



## Create New Columns

In [ ]:
df.withColumn("newAge", df['age']).show()

+----+----+------+
| age|name|newAge|
+----+----+------+
|null| Bob|  null|
|  40| Jim|    40|
|  24|Mary|    24|
+----+----+------+



In [ ]:
df.show()

+----+----+
| age|name|
+----+----+
|null| Bob|
|  40| Jim|
|  24|Mary|
+----+----+



In [ ]:
df.withColumnRenamed("name","firstName").show()

+----+---------+
| age|firstName|
+----+---------+
|null|      Bob|
|  40|      Jim|
|  24|     Mary|
+----+---------+



In [ ]:
df.show()

+----+----+
| age|name|
+----+----+
|null| Bob|
|  40| Jim|
|  24|Mary|
+----+----+



In [ ]:
df.withColumn("agePlusTen", df['age']+10).show()

+----+----+----------+
| age|name|agePlusTen|
+----+----+----------+
|null| Bob|      null|
|  40| Jim|        50|
|  24|Mary|        34|
+----+----+----------+



In [ ]:
df.withColumn("age_minus_5", df['age']-5).show()

+----+----+-----------+
| age|name|age_minus_5|
+----+----+-----------+
|null| Bob|       null|
|  40| Jim|         35|
|  24|Mary|         19|
+----+----+-----------+



## Using SQL

In [ ]:
df.createOrReplaceTempView("custmers")

In [ ]:
sql_results = spark.sql("SELECT * from custmers")

In [ ]:
sql_results

DataFrame[age: int, name: string]

In [ ]:
sql_results.show()

+----+----+
| age|name|
+----+----+
|null| Bob|
|  40| Jim|
|  24|Mary|
+----+----+



In [ ]:
spark.sql("SELECT * FROM custmers WHERE age=24").show()

+---+----+
|age|name|
+---+----+
| 24|Mary|
+---+----+



## DataFrame Operations

- Cover basic operations with Spark DataFrames.
- Use stock data from Walmart.

In [ ]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/WMT.csv >> WMT.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89556  100 89556    0     0   392k      0 --:--:-- --:--:-- --:--:--  392k


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("operations").getOrCreate()
df = spark.read.csv('WMT.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: string (nullable = true)



In [ ]:
df.head(5)

[Row(Date='2016-01-20', Open=61.799999, High=62.330002, Low=60.200001, Close=60.84, Adj Close=53.990601, Volume='17369100'),
 Row(Date='2016-01-21', Open=60.98, High=62.790001, Low=60.91, Close=61.880001, Adj Close=54.913509, Volume='12089200'),
 Row(Date='2016-01-22', Open=62.439999, High=63.259998, Low=62.130001, Close=62.689999, Adj Close=55.632324, Volume='9197500'),
 Row(Date='2016-01-25', Open=62.779999, High=63.82, Low=62.549999, Close=63.450001, Adj Close=56.306763, Volume='12823400'),
 Row(Date='2016-01-26', Open=63.360001, High=64.470001, Low=63.259998, Close=64.0, Adj Close=56.794834, Volume='9441200')]

## Filtering Data

- DataFrames allow for quick filtering of data based on conditions 


In [ ]:
df.filter('Close<62').show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
+----------+---------+---------+---------+---------+---------+--------+



In [ ]:
df.filter('Close<62').select('Open').show()

+---------+
|     Open|
+---------+
|61.799999|
|    60.98|
|61.799999|
|    60.98|
+---------+



In [ ]:
df.filter('Close<62').select(['Date','Open']).show()

+----------+---------+
|      Date|     Open|
+----------+---------+
|2016-01-20|61.799999|
|2016-01-21|    60.98|
|2016-01-20|61.799999|
|2016-01-21|    60.98|
+----------+---------+



## Using Comparison Operators
- Using comparison operators will look similar to SQL operators
- Make to call the entire column within the dataframe

In [ ]:
df.filter(df['Close'] < 62).show()

+----------+---------+---------+---------+---------+---------+--------+
|      Date|     Open|     High|      Low|    Close|Adj Close|  Volume|
+----------+---------+---------+---------+---------+---------+--------+
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
|2016-01-20|61.799999|62.330002|60.200001|    60.84|53.990601|17369100|
|2016-01-21|    60.98|62.790001|    60.91|61.880001|54.913509|12089200|
+----------+---------+---------+---------+---------+---------+--------+



In [ ]:
df.filter((df['Close'] < 62) & ~(df['Open'] > 60)).show()

+----+----+----+---+-----+---------+------+
|Date|Open|High|Low|Close|Adj Close|Volume|
+----+----+----+---+-----+---------+------+
+----+----+----+---+-----+---------+------+



In [ ]:
df.filter(df['Open'] == 60.98).show(1)

+----------+-----+---------+-----+---------+---------+--------+
|      Date| Open|     High|  Low|    Close|Adj Close|  Volume|
+----------+-----+---------+-----+---------+---------+--------+
|2016-01-21|60.98|62.790001|60.91|61.880001|54.913509|12089200|
+----------+-----+---------+-----+---------+---------+--------+
only showing top 1 row



In [ ]:
df.filter(df['Open'] == 60.98).collect()

[Row(Date='2016-01-21', Open=60.98, High=62.790001, Low=60.91, Close=61.880001, Adj Close=54.913509, Volume='12089200'),
 Row(Date='2016-01-21', Open=60.98, High=62.790001, Low=60.91, Close=61.880001, Adj Close=54.913509, Volume='12089200')]

In [ ]:
res =df.filter(df['Open'] == 60.98).collect()

In [ ]:
type(res[0])

pyspark.sql.types.Row

In [ ]:
res[0].asDict()

{'Adj Close': 54.913509,
 'Close': 61.880001,
 'Date': '2016-01-21',
 'High': 62.790001,
 'Low': 60.91,
 'Open': 60.98,
 'Volume': '12089200'}

In [ ]:
for item in res[0]:
  print(item)

2016-01-21
60.98
62.790001
60.91
61.880001
54.913509
12089200


In [ ]:
import pandas as pd

In [ ]:
pd.Series(res[0].asDict())

Date         2016-01-21
Open              60.98
High              62.79
Low               60.91
Close             61.88
Adj Close       54.9135
Volume         12089200
dtype: object

# GroupBy and Aggregate Functions
- `GroupBy` allows you to group rows together based off some column value
- Once you've performed the `GroupBy` operation you can use an aggregate function off that data. 
- An aggregate function aggregates multiple rows of data into a single output, such as taking the sum of inputs, or counting the number of inputs.



In [6]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName("groupbyagg").getOrCreate()

## Import Data


In [11]:
!curl https://raw.githubusercontent.com/markumreed/colab_pyspark/main/sales_data.csv >> sales_data.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   202  100   202    0     0   1836      0 --:--:-- --:--:-- --:--:--  1836


In [12]:
df = spark.read.csv("sales_data.csv", inferSchema=True, header=True)

In [13]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- representative: string (nullable = true)
 |-- num_sales: double (nullable = true)



In [14]:
df.show()

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|    XYZ|           Bob|    200.0|
|    XYZ|           Tom|    120.0|
|    XYZ|         Frank|    340.0|
|   ABCD|         Jerry|    600.0|
|   ABCD|           Amy|    124.0|
|   ABCD|       Vanessa|    243.0|
|     OK|          Carl|    870.0|
|     OK|         Sarah|    350.0|
|   BLAH|          John|    250.0|
|   BLAH|         Linda|    130.0|
|   BLAH|          Mike|    750.0|
|   BLAH|         Chris|    350.0|
+-------+--------------+---------+



## Grouping Data
- Group the data by company

In [15]:
df.groupBy("company")

## Aggregate Functions
- mean, count, max, min, sum...

In [16]:
df.groupBy("company").mean().show()

+-------+-----------------+
|company|   avg(num_sales)|
+-------+-----------------+
|   BLAH|            370.0|
|    XYZ|            220.0|
|     OK|            610.0|
|   ABCD|322.3333333333333|
+-------+-----------------+



In [17]:
df.groupBy("company").count().show()

+-------+-----+
|company|count|
+-------+-----+
|   BLAH|    4|
|    XYZ|    3|
|     OK|    2|
|   ABCD|    3|
+-------+-----+



In [18]:
df.groupBy("company").min().show()

+-------+--------------+
|company|min(num_sales)|
+-------+--------------+
|   BLAH|         130.0|
|    XYZ|         120.0|
|     OK|         350.0|
|   ABCD|         124.0|
+-------+--------------+



In [19]:
df.groupBy("company").max().show()

+-------+--------------+
|company|max(num_sales)|
+-------+--------------+
|   BLAH|         750.0|
|    XYZ|         340.0|
|     OK|         870.0|
|   ABCD|         600.0|
+-------+--------------+



In [20]:
df.groupBy("company").sum().show()

+-------+--------------+
|company|sum(num_sales)|
+-------+--------------+
|   BLAH|        1480.0|
|    XYZ|         660.0|
|     OK|        1220.0|
|   ABCD|         967.0|
+-------+--------------+



## Aggregating

- Not all methods need a groupby call, instead you can just call the generalized `.agg()` method, that will call the aggregate across all rows in the dataframe column specified. 
- It can take in arguments as a single column, or create multiple aggregate calls all at once using dictionary notation.


In [24]:
df.agg({"num_sales":"max"}).show()

+--------------+
|max(num_sales)|
+--------------+
|         870.0|
+--------------+



In [25]:
df.groupBy("company").agg({"num_sales":"mean"}).show()

+-------+-----------------+
|company|   avg(num_sales)|
+-------+-----------------+
|   BLAH|            370.0|
|    XYZ|            220.0|
|     OK|            610.0|
|   ABCD|322.3333333333333|
+-------+-----------------+



In [26]:
company_groups = df.groupBy("company")

In [29]:
company_groups.min().show()

+-------+--------------+
|company|min(num_sales)|
+-------+--------------+
|   BLAH|         130.0|
|    XYZ|         120.0|
|     OK|         350.0|
|   ABCD|         124.0|
+-------+--------------+



## Functions
There are a variety of functions you can import from pyspark.sql.functions.

In [30]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [31]:
df.select(countDistinct("num_sales")).show()

+-------------------------+
|count(DISTINCT num_sales)|
+-------------------------+
|                       11|
+-------------------------+



In [35]:
df.select(avg("num_sales")).show()

+-----------------+
|   avg(num_sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [36]:
df.select(stddev("num_sales")).show()

+----------------------+
|stddev_samp(num_sales)|
+----------------------+
|    250.08742410799007|
+----------------------+



### Alias
- To change the name, use the `.alias()` method for this:

In [34]:
df.select(countDistinct("num_sales").alias("ANYTHING WE WANT")).show()

+----------------+
|ANYTHING WE WANT|
+----------------+
|              11|
+----------------+



### Precision
- Use the `format_number` to change precision


In [37]:
from pyspark.sql.functions import format_number

In [39]:
sales_std = df.select(stddev("num_sales").alias("stddev"))

In [40]:
sales_std.show()

+------------------+
|            stddev|
+------------------+
|250.08742410799007|
+------------------+



In [41]:
sales_std.select(format_number("stddev",2)).show()

+------------------------+
|format_number(stddev, 2)|
+------------------------+
|                  250.09|
+------------------------+



## Order By


In [42]:
df.orderBy("num_sales").show() # Ascending Order

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|    XYZ|           Tom|    120.0|
|   ABCD|           Amy|    124.0|
|   BLAH|         Linda|    130.0|
|    XYZ|           Bob|    200.0|
|   ABCD|       Vanessa|    243.0|
|   BLAH|          John|    250.0|
|    XYZ|         Frank|    340.0|
|     OK|         Sarah|    350.0|
|   BLAH|         Chris|    350.0|
|   ABCD|         Jerry|    600.0|
|   BLAH|          Mike|    750.0|
|     OK|          Carl|    870.0|
+-------+--------------+---------+



In [43]:
df.orderBy(df['num_sales'].desc()).show()

+-------+--------------+---------+
|company|representative|num_sales|
+-------+--------------+---------+
|     OK|          Carl|    870.0|
|   BLAH|          Mike|    750.0|
|   ABCD|         Jerry|    600.0|
|     OK|         Sarah|    350.0|
|   BLAH|         Chris|    350.0|
|    XYZ|         Frank|    340.0|
|   BLAH|          John|    250.0|
|   ABCD|       Vanessa|    243.0|
|    XYZ|           Bob|    200.0|
|   BLAH|         Linda|    130.0|
|   ABCD|           Amy|    124.0|
|    XYZ|           Tom|    120.0|
+-------+--------------+---------+

